# Computer Vision and Advanced Machine Learning area of the project.
### Author: Hugo Iglesias Pombo (2025)
### NO1: EDA and preprocessing of the datasets (MIAS and CBIS-DDMS)

### Importations

Libraries

In [3]:
# Basic data manipulation libraries
import numpy as np
import pandas as pd

# Image processing libraries
import cv2
from PIL import Image
import pydicom  # For DICOM medical images

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# File handling
import os
import glob

Datasets

In [8]:
import os
import pandas as pd
import numpy as np
import pydicom
import matplotlib.pyplot as plt
from skimage import exposure

def load_cbis_ddsm(base_dir, csv_path):
    """
    Carga el dataset CBIS-DDSM y sus metadatos.
    
    Args:
        base_dir: Directorio base con las imágenes DICOM
        csv_path: Ruta al archivo CSV con los metadatos
    
    Returns:
        df: DataFrame con metadatos y rutas a imágenes
    """
    # Cargar el CSV con metadatos
    df = pd.read_csv(csv_path)
    
    # Añadir columna con ruta completa a las imágenes
    df['image_path'] = df.apply(
        lambda row: os.path.join(base_dir, row['image file path']), 
        axis=1
    )
    
    # Verificar que las imágenes existen
    df['exists'] = df['image_path'].apply(os.path.exists)
    print(f"Total de registros: {len(df)}")
    print(f"Imágenes encontradas: {sum(df['exists'])}")
    
    # Filtrar solo las imágenes que existen
    df = df[df['exists']]
    
    return df

def load_dicom_image(path):
    """Carga una imagen DICOM y la convierte a array NumPy."""
    try:
        dcm = pydicom.dcmread(path)
        img = dcm.pixel_array
        
        # Normalizar imagen para visualización
        img = exposure.equalize_hist(img)
        return img, dcm
    except Exception as e:
        print(f"Error cargando {path}: {e}")
        return None, None

# Ejemplo de uso
base_dir = '/Volumes/Proyecto Hugo/breast-cancer-analysis/datasets/CBIS-DDSM'
csv_path = '/Volumes/Proyecto Hugo/breast-cancer-analysis/datasets/CBIS-DDSM/csv/mass_case_description_train_set.csv'

# Cargar dataset
ddsm = load_cbis_ddsm(base_dir, csv_path)

# Mostrar información
print(ddsm.columns)
print(ddsm['pathology'].value_counts())
print(ddsm['breast_density'].value_counts())

# Visualizar algunas imágenes
plt.figure(figsize=(15, 10))
for i, (_, row) in enumerate(df.iloc[:4].iterrows()):
    img, dcm = load_dicom_image(row['image_path'])
    if img is not None:
        plt.subplot(2, 2, i+1)
        plt.imshow(img, cmap='gray')
        plt.title(f"Patología: {row['pathology']}\nDensidad: {row['breast_density']}")
        plt.axis('off')
plt.tight_layout()
plt.show()

Total de registros: 1318
Imágenes encontradas: 0
Index(['patient_id', 'breast_density', 'left or right breast', 'image view',
       'abnormality id', 'abnormality type', 'mass shape', 'mass margins',
       'assessment', 'pathology', 'subtlety', 'image file path',
       'cropped image file path', 'ROI mask file path', 'image_path',
       'exists'],
      dtype='object')
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)


<Figure size 1500x1000 with 0 Axes>

In [5]:
def load_combined_datasets(cbis_base_dir, cbis_csv, mias_base_dir, mias_info):
    """
    Carga y unifica CBIS-DDSM y MIAS en un único DataFrame.
    
    Returns:
        df_combined: DataFrame con metadatos de ambos datasets
    """
    # Cargar CBIS-DDSM
    df_cbis = load_cbis_ddsm(cbis_base_dir, cbis_csv)
    df_cbis['dataset'] = 'CBIS-DDSM'
    
    # Añadir etiqueta binaria
    df_cbis['binary_label'] = df_cbis['pathology'].apply(
        lambda x: 1 if x == 'MALIGNANT' else 0
    )
    
    # Cargar MIAS
    df_mias = load_mias_dataset(mias_base_dir, mias_info)
    df_mias['dataset'] = 'MIAS'
    
    # Añadir etiqueta binaria
    df_mias['binary_label'] = df_mias['severity_name'].apply(
        lambda x: 1 if x == 'Malignant' else 0
    )
    
    # Seleccionar columnas comunes
    df_cbis_selected = df_cbis[['image_path', 'dataset', 'binary_label']]
    df_mias_selected = df_mias[['image_path', 'dataset', 'binary_label']]
    
    # Combinar datasets
    df_combined = pd.concat([df_cbis_selected, df_mias_selected], ignore_index=True)
    
    print(f"Dataset combinado: {len(df_combined)} imágenes")
    print(f"CBIS-DDSM: {len(df_cbis)} imágenes")
    print(f"MIAS: {len(df_mias)} imágenes")
    print(f"Distribución de etiquetas: {df_combined['binary_label'].value_counts()}")
    
    return df_combined

def load_and_preprocess_image(path, target_size=(224, 224)):
    """
    Carga y preprocesa una imagen independiente del formato.
    """
    try:
        if path.lower().endswith('.dcm'):
            # Cargar DICOM
            dcm = pydicom.dcmread(path)
            img = dcm.pixel_array
        else:
            # Cargar otros formatos con PIL
            img = np.array(Image.open(path))
        
        # Convertir a 3 canales si es escala de grises
        if len(img.shape) == 2:
            img = np.stack([img] * 3, axis=-1)
        
        # Redimensionar
        img = transform.resize(img, target_size)
        
        # Normalizar a [0, 1]
        img = img / np.max(img)
        
        return img
    except Exception as e:
        print(f"Error cargando {path}: {e}")
        return None

In [6]:
# Ejemplo de uso
cbis_base_dir = '/Volumes/Proyecto Hugo/breast-cancer-analysis/datasets/CBIS-DDSM'
cbis_csv = '/Volumes/Proyecto Hugo/breast-cancer-analysis/datasets/CBIS-DDSM/csv/mass_case_description_train_set.csv'
mias_base_dir = '/Volumes/Proyecto Hugo/breast-cancer-analysis/datasets/MIAS'
mias_info = '/Volumes/Proyecto Hugo/breast-cancer-analysis/datasets/MIAS/Info.txt'

# Cargar dataset combinado
df_combined = load_combined_datasets(cbis_base_dir, cbis_csv, mias_base_dir, mias_info)

# Visualizar algunas imágenes de ambos datasets
plt.figure(figsize=(15, 10))
for i, dataset in enumerate(['CBIS-DDSM', 'MIAS']):
    # Obtener muestras de cada dataset
    df_subset = df_combined[df_combined['dataset'] == dataset].sample(4)
    
    for j, (_, row) in enumerate(df_subset.iterrows()):
        img = load_and_preprocess_image(row['image_path'])
        if img is not None:
            plt.subplot(2, 4, i*4 + j + 1)
            plt.imshow(img)
            plt.title(f"{dataset}\nLabel: {'Malignant' if row['binary_label'] == 1 else 'Benign'}")
            plt.axis('off')
plt.tight_layout()
plt.show()

NameError: name 'load_cbis_ddsm' is not defined